In [ ]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
mkdir -p data

In [ ]:
# Meli Data
downloaded = drive.CreateFile({'id':"1TpzCRwb4hLMkPwlv1jNy1clpBBg0ajvL"})   # replace the id with id of file you want to access
downloaded.GetContentFile('./data/meli-challenge-2019.tar.bz2')        # replace the file name with your file

# Diccionary
downloaded = drive.CreateFile({'id':"11_zHF4AjPRUjkW_x9CFKJGiPBm6lCKO2"})   # replace the id with id of file you want to access
downloaded.GetContentFile('./data/SBW-vectors-300-min5.txt.bz2')        # replace the file name with your file


In [ ]:
!nvcc --version

In [ ]:
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install --upgrade gensim mlflow tqdm

Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (3.8.3)
Requirement already up-to-date: mlflow in /usr/local/lib/python3.7/dist-packages (1.14.1)
Requirement already up-to-date: tqdm in /usr/local/lib/python3.7/dist-packages (4.59.0)


In [ ]:
%%bash

#mkdir -p data
#curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
#curl -L http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 -o ./data/SBW-vectors-300-min5.txt.bz2
tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/
bunzip2 ./data/SBW-vectors-300-min5.txt.bz2

meli-challenge-2019/
meli-challenge-2019/spanish.train.csv.gz
meli-challenge-2019/portuguese.train.csv.gz
meli-challenge-2019/spanish.test.csv.gz
meli-challenge-2019/portuguese.test.csv.gz


bunzip2: Output file ./data/SBW-vectors-300-min5.txt already exists.


In [ ]:
# para ahicar el dataset de train 
# Acordate de corregir en las funciones de prepocesamiento

%%bash

zcat ./data/meli-challenge-2019/spanish.train.csv.gz | head -n 1000000 | gzip > ./data/meli-challenge-2019/spanish-train-debug.csv.gz

# Librearías

In [ ]:
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from torch.utils.data import IterableDataset, DataLoader
from tqdm.notebook import tqdm, trange

# Dataset

In [ ]:
!zcat ./data/meli-challenge-2019/spanish.train.csv.gz | wc -l

6119101


In [ ]:
import numpy as np

train_index = np.random.rand(6119101) <= 0.8

with gzip.open("./data/meli-challenge-2019/spanish.train.csv.gz", "rt") as spanish_corpus,\
    gzip.open("./data/meli-challenge-2019/spanish-train.csv.gz", "wt") as spanish_train,\
    gzip.open("./data/meli-challenge-2019/spanish-validation.csv.gz", "wt") as spanish_validation:
  for idx, line in enumerate(tqdm(spanish_corpus, total=6119101)):
    if train_index[idx]:
      spanish_train.write(line)
    else:
      spanish_validation.write(line)

  0%|          | 0/6119101 [00:00<?, ?it/s]

In [ ]:
!zcat ./data/meli-challenge-2019/spanish-train.csv.gz | wc -l

4893696


In [ ]:
!zcat ./data/meli-challenge-2019/spanish-validation.csv.gz | wc -l

1225405


In [ ]:
!zcat ./data/meli-challenge-2019/spanish.test.csv.gz | wc -l

63681


In [ ]:
!zcat ./data/meli-challenge-2019/spanish-validation.csv.gz | head

spanish,reliable,Grabadora De Dvd/cd Slim Externa Usb. Lg Mundoe Z/n,DVD_RECORDERS
spanish,reliable,Campana De Cocina Tamel Cristal 90 Cm Tst,KITCHEN_RANGE_HOODS
spanish,reliable,Calefon White Westinhouse,WATER_HEATERS
spanish,reliable,Taylormade Híbrido M1 / 19 Grados Stiff,GOLF_CLUBS_WOODS_AND_HYBRIDS
spanish,reliable,Reloj Nuevo Camara 1.3 Megapixel Bluetooth Smart Inteligente,SMARTWATCHES
spanish,reliable,Parlante Delantero Chevrolet Corsa 2 50w Rms Cuatriaxial Zuk,VEHICLE_SPEAKERS
spanish,reliable,Valija Carry On 55 Cm Delsey Helium Classic,SUITCASES
spanish,reliable,Desgras Complex X 60 Comprimidos,SUPPLEMENTS
spanish,reliable,Puzzle 252 Piezas Reino Animal / Animal Planet Ploppy 563808,PUZZLES
spanish,reliable,Purina Excellent Adult Cat 15kg + Envío Gratis En El Día.,CATS_AND_DOGS_FOODS


In [ ]:
import csv
import random

class MeliChallengeDataset(IterableDataset):
    def __init__(self,
                 dataset_path,
                 dataset_size,
                 random_buffer_size=2048,
                 transform=None):
        assert random_buffer_size > 0
        self.dataset_path = dataset_path
        self.dataset_size = dataset_size
        self.random_buffer_size = random_buffer_size
        self.transform = transform
    
    def __len__(self):
        return self.dataset_size

    def __iter__(self):
        try:
            with gzip.open(self.dataset_path, "rt") as dataset:
                shuffle_buffer = []
                csv_reader = csv.reader(dataset)
                next(csv_reader)

                for line in csv_reader:
                    _, label_quality, title, category = line

                    if self.random_buffer_size == 1:
                        item = {
                            "title": f"{label_quality} {title}",
                            "category": category
                        }
                        if self.transform:
                            item = self.transform(item)
                        yield item
                    else:
                        shuffle_buffer.append({
                            "title": f"{label_quality} {title}",
                            "category": category
                        })

                        if len(shuffle_buffer) == self.random_buffer_size:
                            random.shuffle(shuffle_buffer)
                            for item in shuffle_buffer:
                                if self.transform:
                                    item = self.transform(item)
                                yield item
                            shuffle_buffer = []

            if len(shuffle_buffer) > 0:
                random.shuffle(shuffle_buffer)
                for item in shuffle_buffer:
                    if self.transform:
                        item = self.transform(item)
                    yield item
        except GeneratorExit:
            return

## Preporcesamiento

In [ ]:
class RawDataProcessor:
    def __init__(self, 
                 dataset_path,
                 dataset_size,
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        self.dictionary = corpora.Dictionary()
        labels = set()

        with gzip.open(dataset_path, "rt") as dataset:
            csv_reader = csv.reader(dataset)
            if ignore_header:
              next(csv_reader)
            for line in tqdm(csv_reader, total=dataset_size):
                _, label_quality, title, category = line
                self.dictionary.add_documents(
                    [self._preprocess_string(f"{label_quality} {title}")]
                )
                labels.add(category)
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(labels)
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target, start=1)}

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx.get(target, 0)
    
    def __call__(self, item):
        if isinstance(item, list):
          data = [self.encode_data(i["title"]) for i in item]
          target = [self.encode_target(i["category"]) for i in item]
        else:
          data = self.encode_data(item["title"])
          target = self.encode_target(item["category"])
        
        return {
            "data": data,
            "target": target
        }

## Lectura de datos

In [ ]:
preprocess = RawDataProcessor(
    "./data/meli-challenge-2019/spanish-train-debug.csv.gz",
    1000000
#    preprocessing_buffer_size=100000
)

  0%|          | 0/1000000 [00:00<?, ?it/s]

In [ ]:
train_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish-train-debug.csv.gz",
    1000000,
    random_buffer_size=2048,
    transform=preprocess
)
validation_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish-validation.csv.gz",
    1224416,
    random_buffer_size=1,
    transform=preprocess
)

test_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish.test.csv.gz",
    63681,
    random_buffer_size=1,
    transform=preprocess
)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(validation_dataset)} validation elements")
print(f"Sample train element:\n{next(iter(train_dataset))}")

Datasets loaded with 1000000 training elements and 1224416 validation elements
Sample train element:
{'data': [4, 136, 138, 135, 137], 'target': 56}


In [ ]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.LongTensor(target)
        }

In [ ]:
EPOCHS = 5
filters_count = [2, 3, 4]


pad_sequences = PadSequences(min_length=max(filters_count))
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False,
                          collate_fn=pad_sequences, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)

## Modelo de clasificación

In [ ]:
class MeliChallengeClassifier(nn.Module):
    """An 1D Convulational Neural Network for Sentence Classification."""
    def __init__(self,
                 pretrained_embeddings_path, 
                 dictionary,
                 freeze_embedings = True,
                 vector_size = 300,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=2,
                 dropout=0.5):

        super().__init__()
        # Embedding layer
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with open(pretrained_embeddings_path, "rt") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.vocab_size, self.embed_dim = embeddings_matrix.shape
        self.embedding = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        # Conv Network
        self.conv1d_list = nn.ModuleList([
            nn.Conv1d(in_channels=self.embed_dim,
                      out_channels=num_filters[i],
                      kernel_size=filter_sizes[i])
            for i in range(len(filter_sizes))
        ])
        # Fully-connected layer and Dropout
        self.fc = nn.Linear(np.sum(num_filters), num_classes)
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, input_ids):
        """Perform a forward pass through the network.

        Args:
            input_ids (torch.Tensor): A tensor of token ids with shape
                (batch_size, max_sent_length)

        Returns:
            logits (torch.Tensor): Output logits with shape (batch_size,
                n_classes)
        """

        # Get embeddings from `input_ids`. Output shape: (b, max_len, embed_dim)
        x_embed = self.embedding(input_ids).float()

        # Permute `x_embed` to match input shape requirement of `nn.Conv1d`.
        # Output shape: (b, embed_dim, max_len)
        x_reshaped = x_embed.permute(0, 2, 1)

        # Apply CNN and ReLU. Output shape: (b, num_filters[i], L_out)
        x_conv_list = [F.relu(conv1d(x_reshaped)) for conv1d in self.conv1d_list]

        # Max pooling. Output shape: (b, num_filters[i], 1)
        x_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2])
            for x_conv in x_conv_list]
        
        # Concatenate x_pool_list to feed the fully connected layer.
        # Output shape: (b, sum(num_filters))
        x_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in x_pool_list],
                         dim=1)
        
        x = F.relu(x_fc)
        x = torch.sigmoid(self.output(x))
        return x

In [ ]:
model = MeliChallengeClassifier(pretrained_embeddings_path = "./data/SBW-vectors-300-min5.txt",
                 dictionary = preprocess.dictionary,
                 freeze_embedings=True,
                 filter_sizes=[3, 4, 5],
                 num_filters=[100, 100, 100],
                 num_classes=len(preprocess.target_to_idx) + 1,
                 dropout=0.5)

In [ ]:
print(model)

MeliChallengeClassifier(
  (embedding): Embedding(22851, 300, padding_idx=0)
  (conv1d_list): ModuleList(
    (0): Conv1d(300, 100, kernel_size=(3,), stride=(1,))
    (1): Conv1d(300, 100, kernel_size=(4,), stride=(1,))
    (2): Conv1d(300, 100, kernel_size=(5,), stride=(1,))
  )
  (fc): Linear(in_features=300, out_features=633, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
mlflow.set_experiment("convolutional neural network improved")

with mlflow.start_run():
    mlflow.log_param("model_name", "cnn non-static")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "dropout": 0.2,
        "filters_length": [2, 3, 4],
        "filters_count": 100,
        "learning rate": [0.001]
    })

    model = MeliChallengeClassifier(pretrained_embeddings_path = "./data/SBW-vectors-300-min5.txt",
                 dictionary = preprocess.dictionary,
                 freeze_embedings=True,
                 filter_sizes=[2, 3, 4],
                 num_filters=[100, 100, 100],
                 num_classes=len(preprocess.target_to_idx) + 1,
                 dropout=0.2)
    
    device = torch.device("cuda")
    model = model.to(device)
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(EPOCHS):
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            loss_value = loss(output, target)
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        # Evaluación con los datos de validacion
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        with torch.no_grad():
          for batch in tqdm(validation_loader):
            data = batch["data"].to(device)
            target = batch["target"].to(device)
            output = model(data)
            running_loss.append(
                loss(output, target).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
        mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)

        # Evaluación con los datos de test
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        with torch.no_grad():
          for batch in tqdm(test_loader):
            data = batch["data"].to(device)
            target = batch["target"].to(device)
            output = model(data)
            running_loss.append(
                loss(output, target).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)


    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            targets.extend(target.cpu().detach().numpy())
            predictions.extend(output.squeeze().cpu().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/38243 [00:00<?, ?it/s]

  0%|          | 0/9566 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/38243 [00:00<?, ?it/s]

...

  0%|          | 0/38243 [00:00<?, ?it/s]

  0%|          | 0/9566 [00:00<?, ?it/s]

  0%|          | 0/498 [00:00<?, ?it/s]

  0%|          | 0/38243 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
!tar zcvf mlruns.tar.gz mlruns

In [ ]:
class LSTMClassifier(nn.Module):
	def __init__(self, 
              batch_size, 
              output_size, 
              hidden_size, 
              pretrained_embeddings_path, 
              dictionary,
              num_layers,
              vector_size,
              freeze_embedings):
		super(LSTMClassifier, self).__init__()
		
		
		self.batch_size = batch_size
		self.output_size = output_size
		self.hidden_size = hidden_size
		#self.vocab_size = vocab_size
		#self.embedding_length = embedding_length



		embeddings_matrix = torch.randn(len(dictionary), vector_size)
		embeddings_matrix[0] = torch.zeros(vector_size)
		with open(pretrained_embeddings_path, "rt") as fh:
				 next(fh)
				 for line in fh:
						word, vector = line.strip().split(None, 1)
						if word in dictionary.token2id:
								embeddings_matrix[dictionary.token2id[word]] =\
									torch.FloatTensor([float(n) for n in vector.split()])
		self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix, 
                                                   freeze=freeze_embedings,	
                                                   padding_idx=0)
  
		self.encoder = nn.LSTM(input_size=256, hidden_size=hidden_size, num_layers=num_layers, dropout=0.5,
											 batch_first=True, bidirectional=True)
		self.decoder = nn.Sequential()
		self.drop_en = nn.Dropout(0.6)
		self.bn = nn.BatchNorm1d(hidden_size*2)
		self.decoder.add_module = nn.Linear(hidden_size*2, output_size)
		self.decoder.add_module('softmax',nn.LogSoftmax(dim=-1))

	def forward(self, inputs):
		outputs, _ = self.encoder(inputs)
		predictions = self.decoder(outputs)
		return predictions

In [ ]:
EPOCHS = 1

from torch.autograd import Variable


In [ ]:
mlflow.set_experiment("lstm neural network")

seq_lengths = 64

with mlflow.start_run():
    mlflow.log_param("model_name", "cnn")
    mlflow.log_param("freeze_embedding", True)

    model =  LSTMClassifier(batch_size = 64, 
              output_size = len(preprocess.target_to_idx) + 1, 
              hidden_size = 256, 
              pretrained_embeddings_path = "./data/SBW-vectors-300-min5.txt", 
              dictionary = preprocess.dictionary,
              num_layers = 2,
              vector_size = 300,
              freeze_embedings = True)
    ''''
    model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                    preprocess.dictionary, 
                                    len(preprocess.target_to_idx) + 1,
                                    300, True'
    '''
    device = torch.device("cuda")
    model = model.to(device)
    loss = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), 0.001)
    for epoch in trange(EPOCHS):
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            #retrieve text and no. of words
            text_lengths = batch['data'].shape[1]
            output = model(data)
            loss_value = loss(output, target)
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        # Evaluación con los datos de validacion
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        with torch.no_grad():
          for batch in tqdm(validation_loader):
            data = batch["data"].to(device)
            target = batch["target"].to(device)
            output = model(data)
            running_loss.append(
                loss(output, target).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
        mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)

        # Evaluación con los datos de test
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        with torch.no_grad():
          for batch in tqdm(test_loader):
            data = batch["data"].to(device)
            target = batch["target"].to(device)
            output = model(data)
            running_loss.append(
                loss(output, target).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)


    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            targets.extend(target.cpu().detach().numpy())
            predictions.extend(output.squeeze().cpu().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/157 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
lstm_units = [128, 128]

for idx, units in enumerate(lstm_units[:-1]):
  print(idx)
  print(units)

0
128


In [ ]:
embedding_dim 


(5697, 300)